# Exercise 05

## Demographic Prisoner's Dilemma

The Demographic Prisoner's Dilemma is a family of variants on the classic two-player [Prisoner's Dilemma](https://en.wikipedia.org/wiki/Prisoner's_dilemma), first developed by [Joshua Epstein](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.8.8629&rep=rep1&type=pdf). The model consists of agents, each with a strategy of either Cooperate or Defect, playing against each neighbour in the Moore-neighbourhood. Each agent's payoff is based on its strategy and the strategies of its spatial neighbors. After each step of the model, the agents adopt the strategy of their neighbor (or their own) with the highest total score. 

The specific variant presented here is adapted from the [Evolutionary Prisoner's Dilemma](http://ccl.northwestern.edu/netlogo/models/Prisoner'sDilemmaBasicEvolutionary) model included with NetLogo. Its payoff table is a slight variant of the traditional PD payoff table:

<table>
    <tr><td></td><td>Cooperate</td><td>Defect</td></tr>
    <tr><td>Cooperate</td><td>1, 1</td><td>0, D</td></tr>
    <tr><td>Defect</td><td>D, 0</td><td>0, 0</td></tr>
</table>

Where *D* is the defection bonus, generally set higher than 1. In these runs, the defection bonus is set to $D=1.6$.

The Demographic Prisoner's Dilemma demonstrates how simple rules can lead to the emergence of widespread cooperation, despite the Defection strategy dominating each individual interaction game. However, it is also interesting for another reason: it is known to be sensitive to the activation regime employed in it.

Below, we demonstrate this by instantiating the same model (with the same random seed) three times, with three different activation regimes: 

* Sequential activation, where agents are activated in the order they were added to the model;
* Random activation, where they are activated in random order every step;
* Simultaneous activation, simulating them all being activated simultaneously.



In [ ]:
import sys
sys.path.insert(0,'task02/pd_grid')
from pd_grid.model import PdGrid

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec
from matplotlib.colors import LinearSegmentedColormap
from mesa.visualization import show_logs, freeze_logs

%matplotlib inline
matplotlib.rcParams['font.size'] = 12

## Helper functions

In [ ]:
uniks = LinearSegmentedColormap.from_list( 'unik', [np.array((80,149,200))/255, np.array((74,172,150))/255,
                                                  np.array((234,195,114))/255, np.array((199,16,92))/255])

def draw_grid(model, ax=None):
    """
    Draw the current state of the grid, with Defecting agents in red
    and Cooperating agents in blue.
    """
    if not ax:
        fig, ax = plt.subplots(figsize=(6, 6))
    grid = np.zeros((model.grid.width, model.grid.height))
    for agent, (x, y) in model.grid.coord_iter():
        if agent.move == "D":
            grid[y][x] = 1
        else:
            grid[y][x] = 0
    ax.pcolormesh(grid, cmap=uniks, vmin=0, vmax=1)
    ax.axis("off")
    ax.set_title("Steps: {}".format(model.schedule.steps))

In [ ]:
def draw_score_grid(model, ax=None, minvalue=None, maxvalue=None):
    """
    Draw the current state of the grid, with Defecting agents in red
    and Cooperating agents in blue.
    """
    if maxvalue == None:
        maxvalue = max(model.schedule.agents, key=lambda agent: agent.score).score
    if minvalue == None:
        minvalue = min(model.schedule.agents, key=lambda agent: agent.score).score
    if not ax:
        fig, ax = plt.subplots(figsize=(6, 6))
    grid = np.zeros((model.grid.width, model.grid.height))
    for agent, (x, y) in model.grid.coord_iter():
        grid[y][x] = agent.score
    fig = ax.pcolormesh(grid, vmin=minvalue, vmax=maxvalue)
    ax.axis("off")
    plt.colorbar(fig)
    ax.set_title("Step: {}".format(model.schedule.steps))

In [ ]:
def run_model(model, steps = [10,10,20,20,40], maxscorevalue = None):
    """
    Run an experiment with a given model, and plot the results.
    """
    fig = plt.figure(figsize=(12, 8))

    for i in range(0, len(steps)):
        draw_grid(model, fig.add_subplot(3, len(steps) + 1, i + 1 ))
        draw_score_grid(model, fig.add_subplot(3, len(steps) + 1, i + 2 + len(steps)), maxvalue=maxscorevalue)
        model.run(steps[i])
        
    draw_grid(model, fig.add_subplot(3, len(steps) + 1, i + 2 ))
    draw_score_grid(model, fig.add_subplot(3, len(steps) + 1, i + 3 + len(steps)), maxvalue=maxscorevalue)
    
    ax = fig.add_subplot(3, 1, 3)

    d = model.datacollector.get_model_vars_dataframe()
    d = d * 100 / model.schedule.get_agent_count()
    d.plot(ax=ax, colormap=uniks)
    

In [ ]:
def compare_runs(models):
    data = pd.DataFrame()
    for model in models:
        modeldata = model.datacollector.get_model_vars_dataframe() * 100 / model.schedule.get_agent_count()
        modeldata.columns = [model.schedule_type]
        data = pd.concat([data, modeldata],axis=1)
        
    fig = plt.figure(figsize=(12, 8))
    ax1 = fig.add_subplot(111)
    data.plot(ax=ax1, colormap=uniks)
    fig.savefig("dpd_compare.png")

## Subtask 2.2

Explain the different evolution of cooperators for random and simultaneous activation schemes by exploring the warm-up phase. Why does for some random seeds the number of cooperators go down before it raises, but not for sequential? (< 300 words)

#### Run DPD in browser

In [ ]:
from app import page
page

### Warm-up: Random activation

Passing `printneighbourscore=True` to model initialisation prints score and `printneighbourorder=True` prints move information about the focal (passing `focalpos=(0,0)`) agents' neighbours. The order of neighbours is row-wise (eg. starting with upper left, then upper middle, upper right, middle left aso.)

In [ ]:
show_logs("DPD")
m_random = PdGrid(10, 10, "Random", seed=25)
run_model(m_random, steps = [1,1,1,1,1,10])
freeze_logs("DPD")

### Warm-up: Simultaneous Activation

In [ ]:
show_logs("DPD")
m_simultaneous = PdGrid(10, 10, "Simultaneous", seed=25)
run_model(m_simultaneous, steps = [1,1,1,1,1])
freeze_logs("DPD")

### Compare runs

In [ ]:
matplotlib.rcParams['font.size'] = 12
compare_runs([m_random, m_simultaneous])

## Subtask 2.3

**Discuss the different activation schemes in light of possible model purposes in general and in particular to the DPD. Which one would you chose, and which not (<200 words)?**

## Subtask 3.1
**Make yourself familiar with running batch runs with the Evacuation model (remember the [mesa tutorial](https://mesa.readthedocs.io/en/latest/tutorials/intro_tutorial.html#batch-run)). Why is the `iterations` parameter important?**

## Subtask 3.2

**Conduct batch runs with variations in `cooperation_mean` and `nervousness_mean` and analyse and discuss the results by adopting the provided code in the jupyter notebook. What effect has an extension of the seed-range, e.g. to`range(1,100)`? Which important statistics formula is relevant (<200 words)?**

In [ ]:
from mesa.batchrunner import batch_run
from mesa.visualization import show_logs, freeze_logs
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
sys.path.insert(0,'../../abmodel')

from fire_evacuation.model import FireEvacuation
from fire_evacuation.agent import Human

Note that the simulation may take a few minutes to terminate!

In [ ]:
from IPython.utils import io
        
params = dict(
    floor_size=10,
    human_count=40,
    alarm_believers_prop = 1.0,
    cooperation_mean = np.arange(0.1,0.7,0.1),
    nervousness_mean = np.arange(0.1,0.7,0.1),
    seed = range(1,10),
)

with io.capture_output() as captured:
    show_logs("FireEvacuation")
    results = # implement batch_run() here
    freeze_logs("FireEvacuation")

In [ ]:
pd.DataFrame(results)

In [ ]:
data = pd.DataFrame(results)[['cooperation_mean', 'nervousness_mean','Step']].round(decimals=1)
df = data.groupby(['cooperation_mean', 'nervousness_mean']).agg("mean")
plot_df = (df.reset_index()
              .pivot(index='cooperation_mean', columns='nervousness_mean'))

fig, ax = plt.subplots()
im = ax.imshow(plot_df)
plt.colorbar(im)
plt.xlabel('nervousness_mean')
plt.ylabel('cooperation_mean')
plt.title("Duration of escape")
ax.set_xticks(np.arange(plot_df.shape[1]))
ax.set_xticklabels(sorted(set(round(data['nervousness_mean'], ndigits=1))))
ax.set_yticks(np.arange(plot_df.shape[0]))
ax.set_yticklabels(plot_df.index)
None

## Subtask 3.3

**Formulate a plausible numerical definition of a panic during the evacuation. Consider the proportion of escaped individuals and time. Can you change the output of batch_run to check your criteria (you don’t have t implement this, but your’re free to do so)?**

## Subtask 3.4

**Discuss whether your notion of panic fulfils the criteria for an emergent phenomenon or whether it is imposed.**